In [1]:
import pdfplumber
import re
import json
import os
import PyPDF2


from dotenv import load_dotenv
from neo4j import GraphDatabase
from py2neo import Graph, Node, Relationship
from langchain_openai import AzureChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain.text_splitter import RecursiveCharacterTextSplitter
from typing import List, Dict, Tuple

In [2]:
load_dotenv()
# Recupera endpoint e chiave dalle variabili d'ambiente
api_base = os.getenv("AZURE_OPENAI_API_BASE")
api_key = os.getenv("AZURE_OPENAI_API_KEY")

# neo4j first instance parameters
NEO4J_URI= "neo4j+s://0482640f.databases.neo4j.io"
NEO4J_USERNAME= "neo4j"
NEO4J_PASSWORD= "PNvdaZlk326-ja2hRD1K97ZUUMnD4mj0NsecZNu5-9k"
AURA_INSTANCEID= "0482640f"
AURA_INSTANCENAME= "Instance01"

driver = GraphDatabase.driver(NEO4J_URI, auth=(NEO4J_USERNAME, NEO4J_PASSWORD))

In [3]:
def crea_llm(api_key: str, api_base: str) -> AzureChatOpenAI:
    """
    Crea un'istanza del modello linguistico Azure OpenAI.
    
    Args:
        api_key (str): La chiave API di Azure
        api_base (str): L'URL base dell'API Azure
        
    Returns:
        AzureChatOpenAI: Istanza configurata del modello
    """
    return AzureChatOpenAI(
        openai_api_version="2024-08-01-preview",
        azure_deployment="o1-mini",
        azure_endpoint=api_base,
        api_key=api_key,
        temperature=1
    )

llm = crea_llm(api_key, api_base)

In [ ]:
import json
from typing import List, Dict

def get_database_schema(driver) -> str:
    """Estrae automaticamente lo schema del database."""
    with driver.session() as session:
        # Query per ottenere i tipi di nodi
        node_query = """
        MATCH (n)
        WITH DISTINCT labels(n) AS labels
        RETURN COLLECT(labels) AS types
        """
        
        # Query per ottenere i tipi di relazioni e la loro struttura
        rel_query = """
        MATCH (a)-[r]->(b)
        WITH DISTINCT labels(a) AS sourceLabels, type(r) AS relType, labels(b) AS targetLabels
        RETURN sourceLabels, relType, targetLabels
        """
        
        # Esegui le query separatamente
        node_types = session.run(node_query).single()['types']
        relationships = list(session.run(rel_query))
        
        # Costruisci una descrizione dettagliata dello schema
        schema_description = "Schema del database:\n\nNodi:\n"
        for node_type in node_types:
            schema_description += f"- {':'.join(node_type)}\n"
        
        schema_description += "\nRelazioni e loro struttura:\n"
        for rel in relationships:
            source = ':'.join(rel['sourceLabels'])
            target = ':'.join(rel['targetLabels'])
            schema_description += f"- ({source})-[:{rel['relType']}]->({target})\n"
            
        return schema_description

def generate_cypher_query(llm, schema: str, question: str) -> str:
    """Genera una query Cypher basata sulla domanda dell'utente."""
    prompt = ChatPromptTemplate.from_template("""    
    Sei un esperto di Neo4j. Converti la seguente domanda in una query Cypher.
    
    {schema}
    
    Domanda: {question}
    
    Genera SOLO la query Cypher, senza backticks, senza la parola 'cypher' e senza commenti.
    La query deve usare ESATTAMENTE i nomi delle relazioni come appaiono nello schema.
    IMPORTANTE: Usa SOLO proprietà che sicuramente esistono nello schema. Non inventare proprietà come 'quantità' o 'legale_limite' a meno che non siano esplicitamente menzionate nello schema.
    Assicurati di usare la proprietà 'nome' per fare match sui nomi di nodi. 
    Voglio che ritorni solo il 'numero' del piatto, p.numero
                                              
    """)
    
    messages = prompt.format_messages(
        schema=schema,
        question=question
    )
   # response = llm.invoke(messages)
    response = llm.invoke(messages, temperature=1.0, seed=42)

    # Pulizia della risposta
    query = response.content.strip()
    query = query.replace('```cypher', '').replace('```', '').strip()
    return query

def execute_query(driver, query: str) -> List[Dict]:
    """Esegue la query Cypher e restituisce i risultati grezzi."""
    with driver.session() as session:
        try:
            result = session.run(query)
            print(f"Risultati grezzi della query: {result}")
            
            results = []
            for record in result:
                print(f"Record trovato: {record}")
                
                # Gestisci solo i numeri dei piatti (senza altre informazioni)
                if 'p' in record:
                    node_data = record['p']
                    if isinstance(node_data, dict):
                        # Aggiungi solo il numero del piatto
                        if 'numero' in node_data:
                            results.append({node_data['numero']})
                    else:
                        # Aggiungi solo il numero se è un singolo valore
                        results.append({str(node_data)})

                elif any(k.endswith('.numero') for k in record.keys()):
                    # Se la proprietà è '.numero', estrai il numero
                    nome_key = next(k for k in record.keys() if k.endswith('.numero'))
                    results.append({record[nome_key]})

            return results
        except Exception as e:
            print(f"Errore nell'esecuzione della query: {str(e)}")
            return [{'error': str(e)}]

def query_database(driver, llm, question: str) -> dict:
    """Funzione principale che gestisce l'intero processo di query."""
    try:
        # Ottiene lo schema del database
        schema = get_database_schema(driver)
        #print(f"Schema rilevato:\n{schema}")  # Debug
        
        # Genera la query Cypher
        cypher_query = generate_cypher_query(llm, schema, question)
        print(f"Query Cypher generata: {cypher_query}")  # Debug
        
        # Esegue la query
        results = execute_query(driver, cypher_query)
        
        # Stampa i risultati grezzi come parte del log
        print(f"Risultati grezzi (dati raw): {results}")  # Debug
        
        # Se non ci sono risultati
        if not results:
            return {
                "risultati_grezzi": []
            }
        
        # Restituisce solo i risultati grezzi (numeri dei piatti)
        return {
            "risultati_grezzi": results
        }
        
    except Exception as e:
        # Restituzione di un dizionario anche in caso di errore
        return {
            "risultati_grezzi": [{'error': str(e)}]
        }


In [9]:
import json
from typing import List, Dict

def get_database_schema(driver) -> str:
    """Estrae automaticamente lo schema del database, incluse le proprietà dei nodi e delle relazioni."""
    with driver.session() as session:
        # Query per ottenere i tipi di nodi
        node_query = """
        MATCH (n)
        WITH DISTINCT labels(n) AS labels
        RETURN COLLECT(labels) AS types
        """
        
        # Query per ottenere i tipi di relazioni e la loro struttura
        rel_query = """
        MATCH (a)-[r]->(b)
        WITH DISTINCT labels(a) AS sourceLabels, type(r) AS relType, labels(b) AS targetLabels
        RETURN sourceLabels, relType, targetLabels
        """
        
        # Query per ottenere le proprietà dei nodi
        node_properties_query = """
        MATCH (n)
        WITH DISTINCT labels(n) AS labels, keys(n) AS properties
        UNWIND properties AS property
        RETURN labels, property
        """
        
        # Query per ottenere le proprietà delle relazioni
        rel_properties_query = """
        MATCH ()-[r]->()
        WITH DISTINCT type(r) AS relType, keys(r) AS properties
        UNWIND properties AS property
        RETURN relType, property
        """
        
        # Esegui le query separatamente
        node_types = session.run(node_query).single()['types']
        relationships = list(session.run(rel_query))
        
        # Ottieni le proprietà dei nodi
        node_properties = list(session.run(node_properties_query))
        
        # Ottieni le proprietà delle relazioni
        rel_properties = list(session.run(rel_properties_query))
        
        # Costruisci una descrizione dettagliata dello schema
        schema_description = "Schema del database:\n\nNodi:\n"
        for node_type in node_types:
            schema_description += f"- {':'.join(node_type)}\n"
            # Aggiungi le proprietà dei nodi
            for node_property in node_properties:
                if set(node_type).intersection(set(node_property['labels'])):
                    schema_description += f"    Proprietà: {node_property['property']}\n"
        
        schema_description += "\nRelazioni e loro struttura:\n"
        for rel in relationships:
            source = ':'.join(rel['sourceLabels'])
            target = ':'.join(rel['targetLabels'])
            schema_description += f"- ({source})-[:{rel['relType']}]->({target})\n"
            # Aggiungi le proprietà delle relazioni
            for rel_property in rel_properties:
                if rel_property['relType'] == rel['relType']:
                    schema_description += f"    Proprietà: {rel_property['property']}\n"
            
        return schema_description

def generate_cypher_query(llm, schema: str, question: str) -> str:
    """Genera una query Cypher basata sulla domanda dell'utente."""
    prompt = ChatPromptTemplate.from_template("""    
    Sei un esperto di Neo4j. Converti la seguente domanda in una query Cypher.
    
    {schema}
    
    Domanda: {question}
    
    Genera SOLO la query Cypher, senza backticks, senza la parola 'cypher' e senza commenti.
    La query deve usare ESATTAMENTE i nomi delle relazioni come appaiono nello schema.
    IMPORTANTE: Usa SOLO proprietà che sicuramente esistono nello schema. Non inventare proprietà come 'quantità' o 'legale_limite' a meno che non siano esplicitamente menzionate nello schema.
    Assicurati di usare la proprietà 'nome' per fare match sui nomi di nodi. 
    Voglio che ritorni solo il 'numero' del piatto, p.numero
                                              
    """)
    
    messages = prompt.format_messages(
        schema=schema,
        question=question
    )
    response = llm.invoke(messages, temperature=1.0, seed=42)

    # Pulizia della risposta
    query = response.content.strip()
    query = query.replace('```cypher', '').replace('```', '').strip()
    return query

def execute_query(driver, query: str) -> List[Dict]:
    """Esegue la query Cypher e restituisce i risultati grezzi."""
    with driver.session() as session:
        try:
            result = session.run(query)
            print(f"Risultati grezzi della query: {result}")
            
            results = []
            for record in result:
                print(f"Record trovato: {record}")
                
                # Gestisci solo i numeri dei piatti (senza altre informazioni)
                if 'p' in record:
                    node_data = record['p']
                    if isinstance(node_data, dict):
                        # Aggiungi solo il numero del piatto
                        if 'numero' in node_data:
                            results.append({node_data['numero']})
                    else:
                        # Aggiungi solo il numero se è un singolo valore
                        results.append({str(node_data)})

                elif any(k.endswith('.numero') for k in record.keys()):
                    # Se la proprietà è '.numero', estrai il numero
                    nome_key = next(k for k in record.keys() if k.endswith('.numero'))
                    results.append({record[nome_key]})

            return results
        except Exception as e:
            print(f"Errore nell'esecuzione della query: {str(e)}")
            return [{'error': str(e)}]

def query_database(driver, llm, question: str) -> dict:
    """Funzione principale che gestisce l'intero processo di query."""
    try:
        # Ottiene lo schema del database
        schema = get_database_schema(driver)
        #print(f"Schema rilevato:\n{schema}")  # Debug
        
        # Genera la query Cypher
        cypher_query = generate_cypher_query(llm, schema, question)
        print(f"Query Cypher generata: {cypher_query}")  # Debug
        
        # Esegue la query
        results = execute_query(driver, cypher_query)
        
        # Stampa i risultati grezzi come parte del log
        print(f"Risultati grezzi (dati raw): {results}")  # Debug
        
        # Se non ci sono risultati
        if not results:
            return {
                "risultati_grezzi": []
            }
        
        # Restituisce solo i risultati grezzi (numeri dei piatti)
        return {
            "risultati_grezzi": results
        }
        
    except Exception as e:
        # Restituzione di un dizionario anche in caso di errore
        return {
            "risultati_grezzi": [{'error': str(e)}]
        }


In [7]:
# Esegui la domanda
domanda = "Quali piatti combinano la potenza dello Shard di Materia Oscura con le misteriose Uova di Fenice, utilizzando sia l'Affumicatura Polarizzata a Freddo Iperbarico che la Cottura a Vapore Ecodinamico Bilanciato?"
risposta = query_database(driver, llm, domanda)

# Stampa la risposta
print(f"\nDomanda: {domanda}")
print(f"Risultati grezzi:\n{risposta['risultati_grezzi']}")

Schema rilevato:
Schema del database:

Nodi:
- Macrotecnica
    Proprietà: nome
    Proprietà: descrizione
- Piatto
    Proprietà: nome
    Proprietà: numero
    Proprietà: nome
- Tecnica
    Proprietà: come_funziona
    Proprietà: nome
    Proprietà: vantaggi
    Proprietà: svantaggi
    Proprietà: nome
- Ordine
    Proprietà: nome
    Proprietà: descrizione
- Licenza
    Proprietà: nome
    Proprietà: descrizione
    Proprietà: nome
- Sostanza
    Proprietà: nome
    Proprietà: proprieta
    Proprietà: nome
- Pianeta
    Proprietà: nome
- Ristorante
    Proprietà: nome
    Proprietà: chef

Relazioni e loro struttura:
- (Piatto)-[:PREPARATO_CON_INGREDIENTE]->(Sostanza)
- (Piatto)-[:PREPARATO_CON_TECNICA]->(Tecnica)
- (Tecnica)-[:APPARTIENE_A]->(Macrotecnica)
- (Tecnica)-[:RICHIEDE_LICENZA]->(Licenza)
- (Pianeta)-[:DISTANZA]->(Pianeta)
    Proprietà: km
- (Ristorante)-[:HA_LICENZA]->(Licenza)
- (Ristorante)-[:PREPARATO_DA]->(Piatto)
- (Ristorante)-[:PRESENTE_SU]->(Pianeta)

Query Cyphe

In [10]:
import csv

def read_input_csv(input_file):
    """Legge il file CSV di input e restituisce una lista di domande."""
    with open(input_file, mode='r', encoding='utf-8') as file:
        reader = csv.reader(file)
        questions = [row[0] for row in reader]  # Supponiamo che le domande siano nella prima colonna
    return questions

def write_output_csv(output_file, results):
    """Scrive i risultati nel file CSV di output."""
    with open(output_file, mode='w', encoding='utf-8', newline='') as file:
        writer = csv.writer(file)
        writer.writerow(["row_id", "result"])  # Intestazione del CSV
        for row_id, result in results:
            writer.writerow([row_id, result])

def query_multiple_questions(driver, llm, input_file, output_file):
    """Esegue più domande e salva i risultati in un CSV."""
    # Leggi le domande dal file di input
    questions = read_input_csv(input_file)
    
    # Prepara una lista per i risultati
    results = []
    
    # Esegui la funzione query_database per ogni domanda
    for row_id, question in enumerate(questions, start=1):
        print(f"\nEseguendo la domanda {row_id}: {question}")
        
        # Esegui la query
        risposta = query_database(driver, llm, question)
        
        # Prendi la risposta naturale (risultato finale con i numeri dei piatti)
        result = risposta["risultati_grezzi"]
        
        # Aggiungi il risultato alla lista dei risultati
        results.append((row_id, result))
    
    # Scrivi i risultati nel file CSV di output
    write_output_csv(output_file, results)

# Esempio di utilizzo
input_file = "C:/Users/lparigi/Downloads/letspizza/Hackapizza Dataset/domande.csv"  # Il tuo file CSV di input contenente le domande
output_file = "risultati.csv"  # Il file CSV di output dove scrivere i risultati

query_multiple_questions(driver, llm, input_file, output_file)



Eseguendo la domanda 1: domanda
Query Cypher generata: MATCH (p:Piatto) RETURN p.numero
Risultati grezzi della query: <neo4j._sync.work.result.Result object at 0x000002079A22D670>
Record trovato: <Record p.numero=24>
Record trovato: <Record p.numero=25>
Record trovato: <Record p.numero=26>
Record trovato: <Record p.numero=27>
Record trovato: <Record p.numero=28>
Record trovato: <Record p.numero=29>
Record trovato: <Record p.numero=30>
Record trovato: <Record p.numero=31>
Record trovato: <Record p.numero=32>
Record trovato: <Record p.numero=33>
Record trovato: <Record p.numero=0>
Record trovato: <Record p.numero=1>
Record trovato: <Record p.numero=2>
Record trovato: <Record p.numero=3>
Record trovato: <Record p.numero=4>
Record trovato: <Record p.numero=5>
Record trovato: <Record p.numero=6>
Record trovato: <Record p.numero=7>
Record trovato: <Record p.numero=8>
Record trovato: <Record p.numero=9>
Record trovato: <Record p.numero=10>
Record trovato: <Record p.numero=11>
Record trovato:

Received notification from DBMS server: {severity: WARNING} {code: Neo.ClientNotification.Statement.UnknownPropertyKeyWarning} {category: UNRECOGNIZED} {title: The provided property key is not in the database} {description: One of the property names in your query is not available in the database, make sure you didn't misspell it or that the label is available when you run this statement in your application (the missing property name is: grado)} {position: line: 3, column: 28, offset: 157} for query: 'MATCH (r:Ristorante)-[:PREPARATO_DA]->(p:Piatto)-[:PREPARATO_CON_INGREDIENTE]->(s:Sostanza),\n      (r)-[:HA_LICENZA]->(l:Licenza)\nWHERE l.nome = "LTK" AND l.grado >= 7 AND s.nome = "Spezie Melange"\nRETURN p.numero'


Risultati grezzi della query: <neo4j._sync.work.result.Result object at 0x000002079A25AE40>
Risultati grezzi (dati raw): []

Eseguendo la domanda 59: Quali deliziosi piatti possiamo gustare creati da uno chef con la prestigiosa licenza Q di grado 15 che includono la speziata Nduja Fritta Tanto?
Query Cypher generata: MATCH (r:Ristorante)-[:HA_LICENZA]->(l:Licenza {nome: "Q"}), 
      (r)-[:PREPARATO_DA]->(p:Piatto)-[:PREPARATO_CON_INGREDIENTE]->(s:Sostanza {nome: "Nduja Fritta Tanto"})
RETURN p.numero
Risultati grezzi della query: <neo4j._sync.work.result.Result object at 0x000002079A25A690>
Risultati grezzi (dati raw): []

Eseguendo la domanda 60: Quali sono i piatti creati da chef con licenza t di grado 2 che utilizzano la Cottura Idrodinamica Autoregolante?
Query Cypher generata: MATCH (r:Ristorante)-[:HA_LICENZA]->(l:Licenza {nome: 't', descrizione: 'grado 2'}),
      (r)-[:PREPARATO_DA]->(p:Piatto),
      (p)-[:PREPARATO_CON_TECNICA]->(t:Tecnica {nome: 'Cottura Idrodinamica Autore

Received notification from DBMS server: {severity: WARNING} {code: Neo.ClientNotification.Statement.UnknownPropertyKeyWarning} {category: UNRECOGNIZED} {title: The provided property key is not in the database} {description: One of the property names in your query is not available in the database, make sure you didn't misspell it or that the label is available when you run this statement in your application (the missing property name is: temporale)} {position: line: 5, column: 9, offset: 274} for query: "MATCH (p:Piatto)-[:PREPARATO_CON_INGREDIENTE]->(:Sostanza {nome: 'Farina di Nettuno'}),\n      (p)-[:PREPARATO_CON_TECNICA]->(:Tecnica)-[:RICHIEDE_LICENZA]->(l:Licenza),\n      (r:Ristorante)-[:PREPARATO_DA]->(p),\n      (r)-[:PRESENTE_SU]->(:Pianeta {nome: 'Asgard'})\nWHERE l.temporale > 0\nRETURN p.numero"


Risultati grezzi della query: <neo4j._sync.work.result.Result object at 0x000002079A25ABD0>
Risultati grezzi (dati raw): []

Eseguendo la domanda 81: Quali piatti, preparati in un ristorante su Asgard, richiedono la licenza LTK non base e utilizzano Carne di Xenodonte?
Query Cypher generata: MATCH (p:Piatto)-[:PREPARATO_DA]->(r:Ristorante)-[:PRESENTE_SU]->(:Pianeta {nome: 'Asgard'}),
      (p)-[:PREPARATO_CON_TECNICA]->(:Tecnica)-[:RICHIEDE_LICENZA]->(:Licenza {nome: 'LTK non base'}),
      (p)-[:PREPARATO_CON_INGREDIENTE]->(:Sostanza {nome: 'Carne di Xenodonte'})
RETURN p.numero
Risultati grezzi della query: <neo4j._sync.work.result.Result object at 0x000002079A255160>
Risultati grezzi (dati raw): []

Eseguendo la domanda 82: Quali piatti, che necessitano almeno della licenza P di grado 2 per essere preparati, serviti in un ristorante su Pandora, utilizzano Spore Quantiche?
Query Cypher generata: MATCH (p:Piatto)-[:PREPARATO_CON_INGREDIENTE]->(s:Sostanza {nome: 'Spore Quantiche'}),
  

Received notification from DBMS server: {severity: WARNING} {code: Neo.ClientNotification.Statement.UnknownPropertyKeyWarning} {category: UNRECOGNIZED} {title: The provided property key is not in the database} {description: One of the property names in your query is not available in the database, make sure you didn't misspell it or that the label is available when you run this statement in your application (the missing property name is: grado)} {position: line: 5, column: 9, offset: 290} for query: "MATCH (p:Piatto)-[:PREPARATO_CON_INGREDIENTE]->(s:Sostanza {nome: 'Spore Quantiche'}),\n      (p)-[:PREPARATO_CON_TECNICA]->(t:Tecnica)-[:RICHIEDE_LICENZA]->(l:Licenza {nome: 'P'}),\n      (r:Ristorante)-[:PREPARATO_DA]->(p),\n      (r)-[:PRESENTE_SU]->(pan:Pianeta {nome: 'Pandora'})\nWHERE l.grado >= 2\nRETURN p.numero"


Risultati grezzi della query: <neo4j._sync.work.result.Result object at 0x00000207A558BD10>
Risultati grezzi (dati raw): []

Eseguendo la domanda 83: Quali piatti che necessitano della licenza e+ con un grado minimo di 1 sono serviti su Arrakis e utilizzano l'ingrediente Riso di Cassandra?
Query Cypher generata: MATCH (p:Piatto)-[:PREPARATO_CON_INGREDIENTE]->(s:Sostanza {nome: 'Riso di Cassandra'}),
      (p)-[:PREPARATO_CON_TECNICA]->(t:Tecnica)-[:RICHIEDE_LICENZA]->(l:Licenza {nome: 'e+'}),
      (r:Ristorante)-[:PREPARATO_DA]->(p),
      (r)-[:PRESENTE_SU]->(pl:Pianeta {nome: 'Arrakis'})
RETURN p.numero
Risultati grezzi della query: <neo4j._sync.work.result.Result object at 0x00000207A5626060>
Risultati grezzi (dati raw): []

Eseguendo la domanda 84: Quali piatti sono preparati utilizzando almeno una tecnica di taglio e una di surgelamento secondo il di Sirius Cosmo, ma senza l'uso di Polvere di Crononite?
Query Cypher generata: MATCH (p:Piatto)-[:PREPARATO_CON_TECNICA]->(t1:Tecnica

In [23]:
import re
import pandas as pd
import sys
import os

def convert_set_to_comma_string(cell):
    """
    Converte una stringa che rappresenta un insieme di numeri in una stringa di numeri separati da virgole.
    Se la cella è vuota, restituisce "1".
    """
    if not cell or cell == "[]":
        return "\"1\""  # Restituisce "1" per liste vuote con virgolette
    
    # Estrai i numeri utilizzando un'espressione regolare
    numbers = re.findall(r'\{(\d+|\w+)\}', cell)
    
    # Filtra i valori None
    numbers = [num for num in numbers if num != "None"]
    
    # Restituisci i numeri come stringa separata da virgole, racchiusa tra virgolette
    return f"\"{','.join(numbers)}\""

def process_csv(input_path, output_path=None):
    """Processa un file CSV dato il percorso."""
    # Verificare se il file esiste
    if not os.path.exists(input_path):
        print(f"Errore: Il file '{input_path}' non esiste.")
        return
    
    try:
        # Carica il CSV
        df = pd.read_csv(input_path)
        
        # Converti la colonna 'result'
        if 'result' in df.columns:
            df['result'] = df['result'].apply(convert_set_to_comma_string)
            
            # Poiché abbiamo già aggiunto le virgolette manualmente, usiamo quoting=0 (nessun quoting)
            # Creiamo l'output manualmente per evitare problemi con le virgolette
            output_lines = ["row_id,result"]
            for index, row in df.iterrows():
                output_lines.append(f"{row['row_id']},{row['result']}")
            
            output_text = "\n".join(output_lines)
            
            # Salva il CSV risultante se è specificato un percorso di output
            if output_path:
                with open(output_path, 'w') as f:
                    f.write(output_text)
                print(f"File convertito salvato in: {output_path}")
            
            # Mostra il risultato
            print("\nContenuto del CSV convertito:")
            print(output_text)
        else:
            print("Errore: La colonna 'result' non è presente nel CSV.")
    
    except Exception as e:
        print(f"Errore durante l'elaborazione del file: {str(e)}")


        
process_csv(input_path, output_path)

File convertito salvato in: test.csv

Contenuto del CSV convertito:
row_id,result
1,"24,25,26,27,28,29,30,31,32,33,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239,240,241,242,243,244,245,246,247,248,249,250,251,252,253,254,255,2